In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:6 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2021-11-22 03:51:46--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  --.-KB/s    in 0.1s    

2021-11-22 03:51:47 (5.99 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
# Load in employee.csv from S3 into a DataFrame
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Video_Games_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   12039526| RTIS3L2M1F5SM|B001CXYMFS|     737716809|Thrustmaster T-Fl...|     Video Games|          5|            0|          0|   N|                Y|an amazing joysti...|Used this for Eli...| 2015-08-31|
|         US|    9636577| R1ZV7R40OLHKD|B00M920ND6|     569686175|Tonsee 6 buttons ...|     Video Games|          5|    

In [5]:
print((df.count(), len(df.columns)))

(1785997, 15)


## Drop duplicates and incomplete rows

In [6]:
df = df.drop_duplicates()
df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   26197704|R10010D6B1QM7U|B004HX5OFW|       8155204|Xbox 360 Slim AC ...|     Video Games|          1|            0|          0|   N|                Y|                 POS|I purchased this ...| 2013-03-28|
|         US|   10409381|R100BI61LIKH2N|B000QW9D14|     583724133|Blue Dragon - Xbo...|     Video Games|          5|    

In [7]:
print((df.count(), len(df.columns)))

(1785997, 15)


In [8]:
df = df.dropna()
df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   26197704|R10010D6B1QM7U|B004HX5OFW|       8155204|Xbox 360 Slim AC ...|     Video Games|          1|            0|          0|   N|                Y|                 POS|I purchased this ...| 2013-03-28|
|         US|   10409381|R100BI61LIKH2N|B000QW9D14|     583724133|Blue Dragon - Xbo...|     Video Games|          5|    

In [9]:
print((df.count(), len(df.columns)))

(1785886, 15)


## Examine the schema

In [10]:
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



review_date column is a string and needs to be changed to date format before load

## Create a new DataFrames for load process

In [11]:
from pyspark.sql.functions import col
from pyspark.sql.functions import to_date

# Create the review_id DataFrame. 
# Convert the 'review_date' column to a date datatype
review_id = df.select(["review_id","customer_id","product_id","product_parent", to_date("review_date", 'yyyy-MM-dd').alias("review_date")])
review_id.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R10010D6B1QM7U|   26197704|B004HX5OFW|       8155204| 2013-03-28|
|R100BI61LIKH2N|   10409381|B000QW9D14|     583724133| 2013-10-28|
|R100TPJCYDYGBU|   39071377|B00D3IKP7Y|     378642975| 2013-12-31|
|R1018OO9U6UUWO|   32826337|B0094X28J0|     272196441| 2014-03-21|
|R102YIWZEIAXT9|   16385120|B000UQAUWW|     694091073| 2007-11-29|
|R1034S7FA21OI1|   14198390|B004S5PBM0|     275810674| 2011-11-06|
|R103LC8CTAHWZ6|   16965219|B002EE7OKE|     852505908| 2013-10-07|
|R103PT7S4HRIP8|   48295356|B003R7H5TC|     318359987| 2010-12-28|
|R104RCQZYPYDXS|   21497984|B000R3BNDI|     318965716| 2008-02-28|
|R105KZIXHZW63W|   22595160|B000NKFEZI|     256274221| 2013-02-04|
|R105W3ID5NHQ48|   43562425|B002I0JNK8|     930811126| 2011-07-07|
|R106J0FROF1O34|   50248806|B0050SWUTQ|     861146594| 2014-03

In [12]:
# Create the products DataFrame and drop duplicates to contain only unique values. 
products = df.select(["product_id", "product_title"]).drop_duplicates()
products.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00CJ7IUI6|The Elder Scrolls...|
|B00DHF39KS|Wolfenstein: The ...|
|B00MUTAVH6|Under Night In-Bi...|
|B001AZSEUW|              Peggle|
|B00KVOVBGM|PlayStation 4 Con...|
|B00O9VGH4Y|USPRO&reg; Headph...|
|B004OQNZY4|Phineas and Ferb:...|
|B00ZLN980O|Donop seablue 2.4...|
|B002L8W5V6|Dotop Nintendo Ga...|
|B007AJZ5PY|Nyko Game Case fo...|
|B000AOEU2K|Fire Emblem: Path...|
|B000H8BW7U|Tanarus (PC) (Com...|
|B013RADQOQ|Susenstone® 2400D...|
|B00KQXKUJ2|FIFA 15 (Ultimate...|
|B006W41X2C|Turtle Beach - Ea...|
|B000KCX9M4|Grand Theft Auto:...|
|B00YT90JWC|Red Wii Mini Cons...|
|B0096KG6A8|Wii U Super Mario...|
|B00L6AVLB0|World of Tanks-X3...|
|B000IMYKQ0|Wii Nunchuk Contr...|
+----------+--------------------+
only showing top 20 rows



In [13]:
# Create the customers DataFrame and rename column to customer_count
customers = df.groupby("customer_id").agg({"customer_id":"count"}).withColumnRenamed("count(customer_id)", "customer_count")
customers.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   40504830|             8|
|    3675423|             3|
|   11393715|             9|
|   14175902|            22|
|   39664150|             4|
|   14534785|             1|
|   47765241|             7|
|   33929721|             1|
|   41446882|             8|
|   52714119|             4|
|   27910865|             1|
|   31564043|            24|
|   50722423|             2|
|   26206997|             1|
|   21130353|             1|
|   28184372|             1|
|   13005988|            16|
|   13673487|             1|
|   16100357|             1|
|   16863015|             1|
+-----------+--------------+
only showing top 20 rows



In [14]:
# Create the vine DataFrame
vine = df.select(["review_id","star_rating","helpful_votes","total_votes","vine"])
vine.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R10010D6B1QM7U|          1|            0|          0|   N|
|R100BI61LIKH2N|          5|            1|          2|   N|
|R100TPJCYDYGBU|          3|            1|          2|   N|
|R1018OO9U6UUWO|          5|            0|          0|   N|
|R102YIWZEIAXT9|          5|            2|          4|   N|
|R1034S7FA21OI1|          5|            2|          2|   N|
|R103LC8CTAHWZ6|          2|            1|          1|   N|
|R103PT7S4HRIP8|          2|            0|          0|   N|
|R104RCQZYPYDXS|          5|            7|          8|   N|
|R105KZIXHZW63W|          5|            1|          1|   N|
|R105W3ID5NHQ48|          3|            8|          9|   N|
|R106J0FROF1O34|          4|            0|          0|   N|
|R10770OX7D4UYW|          5|            0|          0|   N|
|R1077Z2P6K2RHR|          5|            

## Write DataFrame to RDS

In [15]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://database-01.cvudwnrntrve.us-west-2.rds.amazonaws.com:5432/my_data_class_db_alfred"
config = {"user":"", 
          "password": "", 
          "driver":"org.postgresql.Driver"}

In [20]:
# Write review_id to table in RDS
review_id.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [21]:
# Write products to table in RDS
products.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [22]:
# Write customers to table in RDS
customers.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [23]:
# Write vine to table in RDS
vine.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)